<a href="https://colab.research.google.com/github/dbtjr1103/Dacon_practice/blob/main/nfl_player_contact_detection_helmet_track_ftrs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

V6 : Instead of using all train data (4M 721 K row), we can exclude rows that have a distance greater than 2, they are most likely to have a negative contact ( + 0.99 recall), the new train data have 660 K row, now we can add more features like the difference between player 1 and 2, product and multiple clusters ...



V8 : Handling missing values for The Ground 

In [ ]:
# config class 정의 
import os
import torch

class Config:
    # 코드 작성한 사람의 이름 - 변경하면될듯
    AUTHOR = "colum2131"
    # 실험의 이름 - 변경 ok
    NAME = "NFLC-" + "Exp001-simple-xgb-baseline"
    # 경연대회 이름
    COMPETITION = "nfl-player-contact-detection"
    # 실험에서 사용할 난수 시드
    seed = 42
    # 교차 검증 폴드 수
    num_fold = 5
    # xgboost 모델에서 사용할 하이퍼 파라미터 딕셔너리 
    xgb_params = {
        # 이진분류 문제에 적합한 binary:logistic
        'objective': 'binary:logistic',
        # 모델 평가 지표 - auc
        'eval_metric': 'auc',
        # lr
        'learning_rate':0.03,
        # 사용 가능한경우 gpu 가속을 사용, gpu를 사용할 수 없는경우 hist를 사용한다..
        'tree_method':'hist' if not torch.cuda.is_available() else 'gpu_hist'
    }

In [ ]:
import os
# 가비지컬렉션 기능 제어하기 위한 라이브러리 
import gc 
# 외부 프로세스를 실행하고 제어하는 라이브러리
import subprocess

# numpy 수치계산
import numpy as np
# 판다스 데이터 프레임 
import pandas as pd
# 차트, 플롯 그래픽 시각화 
import matplotlib.pyplot as plt
# 직사각형, 원 등 도형 클래스 제공
import matplotlib.patches as patches
# jupyter 노트북에서 비디오 표시하기 위한 클래스
from IPython.display import Video, display
# 다양한 최적화 알고리즘을 사용해 스칼라 함수를 최소화?
from scipy.optimize import minimize
# 이미지 처리 
import cv2
# 파일 검색 기능
from glob import glob
# 갓큐디엠
from tqdm import tqdm

# k-fold 교차 검증을 위한 클래스
from sklearn.model_selection import GroupKFold
# 머신러닝 모델
from sklearn.metrics import (
    roc_auc_score,
    matthews_corrcoef,
)

# 그레디언트 부스팅을 위한 라이브러리
import xgboost as xgb

import torch

# cupy : gpu 가속수치계산(like numpy), cudf gpu 데이터프레임, 결정포레스트를 사용하여 추론을 수행하기 위한 클래스
if torch.cuda.is_available():
    import cupy 
    import cudf
    from cuml import ForestInference

In [ ]:
def setup(cfg):
    # gpu 지원 가능/불가능
    cfg.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # set dirs : 구체적인 디렉토리 경로 설정
    cfg.INPUT = f'../input/{cfg.COMPETITION}'
    cfg.EXP = cfg.NAME
    cfg.OUTPUT_EXP = cfg.NAME
    cfg.SUBMISSION = './'
    cfg.DATASET = '../input/'

    # 훈련 및 예측파일 저장 경로 설정
    cfg.EXP_MODEL = os.path.join(cfg.EXP, 'model')
    cfg.EXP_FIG = os.path.join(cfg.EXP, 'fig')
    cfg.EXP_PREDS = os.path.join(cfg.EXP, 'preds')

    # make dirs
    for d in [cfg.EXP_MODEL, cfg.EXP_FIG, cfg.EXP_PREDS]:
        os.makedirs(d, exist_ok=True)
        
    # 업데이트 된 cfg 개체 반환    
    return cfg

In [ ]:
# ==============================
# function
# ==============================
# ref: https://www.kaggle.com/code/robikscube/nfl-player-contact-detection-getting-started
# 함수 정의
# 'game_play', 'step', 'nfl_player_id_1', 'nfl_player_id_2' 컬럼을 이용하여 'contact_id'라는 새로운 컬럼을 생성하는 함수
def add_contact_id(df):
    # Create contact ids
    df["contact_id"] = (
        df["game_play"]
        + "_"
        + df["step"].astype("str")
        + "_"
        + df["nfl_player_id_1"].astype("str")
        + "_"
        + df["nfl_player_id_2"].astype("str")
    )
    return df
#  'contact_id' 컬럼을 'game_play', 'step', 'nfl_player_id_1', 'nfl_player_id_2'로 분할하여 데이터프레임에 추가
def expand_contact_id(df):
    """
    Splits out contact_id into seperate columns.
    """
    df["game_play"] = df["contact_id"].str[:12]
    df["step"] = df["contact_id"].str.split("_").str[-3].astype("int")
    df["nfl_player_id_1"] = df["contact_id"].str.split("_").str[-2]
    df["nfl_player_id_2"] = df["contact_id"].str.split("_").str[-1]
    return df

# cross validation
# GroupKFold를 이용하여 데이터셋을 cross-validation하기 위한 fold들을 생성
def get_groupkfold(train, target_col, group_col, n_splits):
    kf = GroupKFold(n_splits=n_splits)
    generator = kf.split(train, train[target_col], train[group_col])
    fold_series = []
    for fold, (idx_train, idx_valid) in enumerate(generator):
        fold_series.append(pd.Series(fold, index=idx_valid))
    fold_series = pd.concat(fold_series).sort_index()
    return fold_series

# xgboost code
# xgboost 모델 학습. cfg.folds를 이용하여 fold를 구분하고, 학습된 모델을 저장하고 cross-validation에 대한 결과를 반환
def fit_xgboost(cfg, X, y, params, add_suffix=''):
    """
    xgb_params = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'learning_rate':0.01,
        'tree_method':'gpu_hist'
    }
    """
    oof_pred = np.zeros(len(y), dtype=np.float32)
    for fold in sorted(cfg.folds.unique()):
        if fold == -1: continue
        idx_train = (cfg.folds!=fold)
        idx_valid = (cfg.folds==fold)
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]
        display(pd.Series(y_valid).value_counts())

        xgb_train = xgb.DMatrix(x_train, label=y_train)
        xgb_valid = xgb.DMatrix(x_valid, label=y_valid)
        evals = [(xgb_train,'train'),(xgb_valid,'eval')]

        model = xgb.train(
            params,
            xgb_train,
            num_boost_round=10_000,
            early_stopping_rounds=100,
            evals=evals,
            verbose_eval=100,
        )

        model_path = os.path.join(cfg.EXP_MODEL, f'xgb_fold{fold}{add_suffix}.model')
        model.save_model(model_path)
        if not torch.cuda.is_available():
            model = xgb.Booster().load_model(model_path)
        else:
            model = ForestInference.load(model_path, output_class=True, model_type='xgboost')
        pred_i = model.predict_proba(x_valid)[:, 1]
        oof_pred[x_valid.index] = pred_i
        score = round(roc_auc_score(y_valid, pred_i), 5)
        print(f'Performance of the prediction: {score}\n')
        del model; gc.collect()

    np.save(os.path.join(cfg.EXP_PREDS, f'oof_pred{add_suffix}'), oof_pred)
    score = round(roc_auc_score(y, oof_pred), 5)
    print(f'All Performance of the prediction: {score}')
    return oof_pred

# 학습된 xgboost 모델을 불러와서 예측, 모델의 평균 예측값을 반환
def pred_xgboost(X, data_dir, add_suffix=''):
    models = glob(os.path.join(data_dir, f'xgb_fold*{add_suffix}.model'))
    if not torch.cuda.is_available():
         models = [xgb.Booster().load_model(model_path) for model in models]
    else:
        models = [ForestInference.load(model, output_class=True, model_type='xgboost') for model in models]
    preds = np.array([model.predict_proba(X)[:, 1] for model in models])
    preds = np.mean(preds, axis=0)
    return preds

In [ ]:
# ==============================
# read data
# ==============================
# config class 객체를 setup 함수에 전달해서 설정 초기화
cfg = setup(Config)

# gpu 가능여부에 따라 df읽어오는 방식? 다르게
if not torch.cuda.is_available():
    # 플레이어 추적 정보 (?)
    tr_tracking = pd.read_csv(os.path.join(cfg.INPUT, 'train_player_tracking.csv'), parse_dates=["datetime"])
    te_tracking = pd.read_csv(os.path.join(cfg.INPUT, 'test_player_tracking.csv'), parse_dates=["datetime"])
    # tr_helmets = pd.read_csv(os.path.join(cfg.INPUT, 'train_baseline_helmets.csv'))
    # te_helmets = pd.read_csv(os.path.join(cfg.INPUT, 'test_baseline_helmets.csv'))
    # 메타데이터 파일?
    # tr_video_metadata = pd.read_csv(os.path.join(cfg.INPUT, 'train_video_metadata.csv'))
    # te_video_metadata = pd.read_csv(os.path.join(cfg.INPUT, 'test_video_metadata.csv'))
    # 샘플 제출 데이터
    sub = pd.read_csv(os.path.join(cfg.INPUT, 'sample_submission.csv'))

    # 학습 데이터 타겟 변수
    train = pd.read_csv(os.path.join(cfg.INPUT, 'train_labels.csv'), parse_dates=["datetime"])
    # 테스트 데이터의 인덱스 정보 (샘플 제출 데이터를 확장하여 생성?)
    test = expand_contact_id(sub)
    
else:
    tr_tracking = cudf.read_csv(os.path.join(cfg.INPUT, 'train_player_tracking.csv'), parse_dates=["datetime"])
    te_tracking = cudf.read_csv(os.path.join(cfg.INPUT, 'test_player_tracking.csv'), parse_dates=["datetime"])
    # tr_helmets = cudf.read_csv(os.path.join(cfg.INPUT, 'train_baseline_helmets.csv'))
    # te_helmets = cudf.read_csv(os.path.join(cfg.INPUT, 'test_baseline_helmets.csv'))
    # tr_video_metadata = cudf.read_csv(os.path.join(cfg.INPUT, 'train_video_metadata.csv'))
    # te_video_metadata = cudf.read_csv(os.path.join(cfg.INPUT, 'test_video_metadata.csv'))
    sub = pd.read_csv(os.path.join(cfg.INPUT, 'sample_submission.csv'))

    train = cudf.read_csv(os.path.join(cfg.INPUT, 'train_labels.csv'), parse_dates=["datetime"])
    test = cudf.DataFrame(expand_contact_id(sub))

The following code is used to create the features.  
Basically, the numerical features contained in player_tracking.csv are merged into player_id_1 and player_id_2 respectively.

In [ ]:
# ==============================
# feature engineering
# ==============================
def create_features(df, tr_tracking, merge_col="step", use_cols=["x_position", "y_position"]):
    output_cols = []
    df_combo = (
        df.astype({"nfl_player_id_1": "str"})
        .merge(
            tr_tracking.astype({"nfl_player_id": "str"})[
                ["game_play", merge_col, "nfl_player_id",] + use_cols
            ],
            left_on=["game_play", merge_col, "nfl_player_id_1"],
            right_on=["game_play", merge_col, "nfl_player_id"],
            how="left",
        )
        .rename(columns={c: c+"_1" for c in use_cols})
        .drop("nfl_player_id", axis=1)
        .merge(
            tr_tracking.astype({"nfl_player_id": "str"})[
                ["game_play", merge_col, "nfl_player_id"] + use_cols
            ],
            left_on=["game_play", merge_col, "nfl_player_id_2"],
            right_on=["game_play", merge_col, "nfl_player_id"],
            how="left",
        )
        .drop("nfl_player_id", axis=1)
        .rename(columns={c: c+"_2" for c in use_cols})
        .sort_values(["game_play", merge_col, "nfl_player_id_1", "nfl_player_id_2"])
        .reset_index(drop=True)
    )
    output_cols += [c+"_1" for c in use_cols]
    output_cols += [c+"_2" for c in use_cols]
    
    if ("x_position" in use_cols) & ("y_position" in use_cols):
        index = df_combo['x_position_2'].notnull()
        if torch.cuda.is_available():
            index = index.to_array()
        distance_arr = np.full(len(index), np.nan)
        tmp_distance_arr = np.sqrt(
            np.square(df_combo.loc[index, "x_position_1"] - df_combo.loc[index, "x_position_2"])
            + np.square(df_combo.loc[index, "y_position_1"]- df_combo.loc[index, "y_position_2"])
        )
        if torch.cuda.is_available():
            tmp_distance_arr = tmp_distance_arr.to_array()
        distance_arr[index] = tmp_distance_arr
        df_combo['distance'] = distance_arr
        output_cols += ["distance"]
        
    df_combo['G_flug'] = (df_combo['nfl_player_id_2']=="G")
    output_cols += ["G_flug"]
    return df_combo, output_cols


use_cols = [
    'x_position', 'y_position', 'speed', 'distance',
    'direction', 'orientation', 'acceleration', 'sa'
]
train, feature_cols = create_features(train, tr_tracking, use_cols=use_cols)
test, feature_cols = create_features(test, te_tracking, use_cols=use_cols)
if torch.cuda.is_available():
    train = train.to_pandas()
    test = test.to_pandas()

display(train)

,contact_id,game_play,datetime,step,nfl_player_id_1,nfl_player_id_2,contact,x_position_1,y_position_1,speed_1,...,x_position_2,y_position_2,speed_2,distance_2,direction_2,orientation_2,acceleration_2,sa_2,distance,G_flug
0,58168_003392_0_37084_37211,58168_003392,2020-09-11 03:01:48.100,0,37084,37211,0,41.90,20.08,0.54,...,39.59,17.07,0.53,0.05,134.84,84.73,1.43,1.42,3.794232,False
1,58168_003392_0_37084_38556,58168_003392,2020-09-11 03:01:48.100,0,37084,38556,0,41.90,20.08,0.54,...,41.93,30.61,0.67,0.05,232.50,227.00,1.82,1.61,10.530043,False
2,58168_003392_0_37084_38567,58168_003392,2020-09-11 03:01:48.100,0,37084,38567,0,41.90,20.08,0.54,...,40.37,19.88,0.66,0.07,136.70,88.92,0.90,0.89,1.543017,False
3,58168_003392_0_37084_38590,58168_003392,2020-09-11 03:01:48.100,0,37084,38590,0,41.90,20.08,0.54,...,40.33,25.28,0.52,0.06,141.08,100.37,0.59,0.58,5.431841,False
4,58168_003392_0_37084_39947,58168_003392,2020-09-11 03:01:48.100,0,37084,39947,0,41.90,20.08,0.54,...,40.11,26.73,0.99,0.09,163.38,90.69,1.68,1.64,6.886697,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4721613,58582_003121_91_52500_52619,58582_003121,2021-10-12 02:42:29.100,91,52500,52619,0,58.74,40.11,1.34,...,58.90,22.09,1.32,0.14,74.21,47.63,0.96,-0.44,18.020710,False
4721614,58582_003121_91_52500_G,58582_003121,2021-10-12 02:42:29.100,91,52500,G,0,58.74,40.11,1.34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4721615,58582_003121_91_52609_52619,58582_003121,2021-10-12 02:42:29.100,91,52609,52619,0,60.32,25.93,1.38,...,58.90,22.09,1.32,0.14,74.21,47.63,0.96,-0.44,4.094142,False
4721616,58582_003121_91_52609_G,58582_003121,2021-10-12 02:42:29.100,91,52609,G,0,60.32,25.93,1.38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


# Exclude distance > 2
if the distance between two players is greater than 2 then the probability of contact is so low, we will consider it = 0, training data will be reduced from 4.7 M rows to 660 K

In [ ]:
DISTANCE_THRESH = 2

train_y = train['contact'].values
oof_pred = np.zeros(len(train))
cond_dis_train = (train['distance']<=DISTANCE_THRESH) | (train['distance'].isna())
cond_dis_test = (test['distance']<=DISTANCE_THRESH) | (test['distance'].isna())

train = train[cond_dis_train]
train.reset_index(inplace = True, drop = True)

print('number of train data : ',len(train))

_ = gc.collect()

number of train data :  660560


# Helmet track Features

In [ ]:
CLUSTERS = [10, 50, 100, 500]

def add_step_pct(df, cluster):
    df['step_pct'] = cluster * (df['step']-min(df['step']))/(max(df['step'])-min(df['step']))
    df['step_pct'] = df['step_pct'].apply(np.ceil).astype(np.int32)
    return df

for cluster in CLUSTERS:
    train = train.groupby('game_play').apply(lambda x:add_step_pct(x,cluster))
    test = test.groupby('game_play').apply(lambda x:add_step_pct(x,cluster))

    for helmet_view in ['Sideline', 'Endzone']:
        helmet_train = pd.read_csv('/kaggle/input/nfl-player-contact-detection/train_baseline_helmets.csv')
        helmet_train.loc[helmet_train['view']=='Endzone2','view'] = 'Endzone'
        helmet_test = pd.read_csv('/kaggle/input/nfl-player-contact-detection/test_baseline_helmets.csv')
        helmet_test.loc[helmet_test['view']=='Endzone2','view'] = 'Endzone'

        helmet_train.rename(columns = {'frame': 'step'}, inplace = True)
        helmet_train = helmet_train.groupby('game_play').apply(lambda x:add_step_pct(x,cluster))
        helmet_test.rename(columns = {'frame': 'step'}, inplace = True)
        helmet_test = helmet_test.groupby('game_play').apply(lambda x:add_step_pct(x,cluster))
        helmet_train = helmet_train[helmet_train['view']==helmet_view]
        helmet_test = helmet_test[helmet_test['view']==helmet_view]

        helmet_train['helmet_id'] = helmet_train['game_play'] + '_' + helmet_train['nfl_player_id'].astype(str) + '_' + helmet_train['step_pct'].astype(str)
        helmet_test['helmet_id'] = helmet_test['game_play'] + '_' + helmet_test['nfl_player_id'].astype(str) + '_' + helmet_test['step_pct'].astype(str)

        helmet_train = helmet_train[['helmet_id', 'left', 'width', 'top', 'height']].groupby('helmet_id').mean().reset_index()
        helmet_test = helmet_test[['helmet_id', 'left', 'width', 'top', 'height']].groupby('helmet_id').mean().reset_index()
        for player_ind in [1, 2]:
            train['helmet_id'] = train['game_play'] + '_' + train['nfl_player_id_'+str(player_ind)].astype(str) + \
                                    '_' + train['step_pct'].astype(str)
            test['helmet_id'] = test['game_play'] + '_' + test['nfl_player_id_'+str(player_ind)].astype(str) + \
                                    '_' + test['step_pct'].astype(str)

            train = train.merge(helmet_train, how = 'left')
            test = test.merge(helmet_test, how = 'left')

            train.rename(columns = {i:i+'_'+helmet_view+'_'+str(cluster)+'_'+str(player_ind) for i in ['left', 'width', 'top', 'height']}, inplace = True)
            test.rename(columns = {i:i+'_'+helmet_view+'_'+str(cluster)+'_'+str(player_ind) for i in ['left', 'width', 'top', 'height']}, inplace = True)

            del train['helmet_id'], test['helmet_id']
            gc.collect()

            feature_cols += [i+'_'+helmet_view+'_'+str(cluster)+'_'+str(player_ind) for i in ['left', 'width', 'top', 'height']]
        del helmet_train, helmet_test
        gc.collect()

# Fill missing values for the ground

In [ ]:
for cluster in CLUSTERS:
    for helmet_view in ['Sideline', 'Endzone']:
        train.loc[train['G_flug']==True,'left_'+helmet_view+'_'+str(cluster)+'_2'] = train.loc[train['G_flug']==True,'left_'+helmet_view+'_'+str(cluster)+'_1']
        train.loc[train['G_flug']==True,'top_'+helmet_view+'_'+str(cluster)+'_2'] = train.loc[train['G_flug']==True,'top_'+helmet_view+'_'+str(cluster)+'_1']
        train.loc[train['G_flug']==True,'width_'+helmet_view+'_'+str(cluster)+'_2'] = 0
        train.loc[train['G_flug']==True,'height_'+helmet_view+'_'+str(cluster)+'_2'] = 0
        
        test.loc[test['G_flug']==True,'left_'+helmet_view+'_'+str(cluster)+'_2'] = test.loc[test['G_flug']==True,'left_'+helmet_view+'_'+str(cluster)+'_1']
        test.loc[test['G_flug']==True,'top_'+helmet_view+'_'+str(cluster)+'_2'] = test.loc[test['G_flug']==True,'top_'+helmet_view+'_'+str(cluster)+'_1']
        test.loc[test['G_flug']==True,'width_'+helmet_view+'_'+str(cluster)+'_2'] = 0
        test.loc[test['G_flug']==True,'height_'+helmet_view+'_'+str(cluster)+'_2'] = 0

# Diffrence & Product features

In [ ]:
cols = [i[:-2] for i in train.columns if i[-2:]=='_1' and i!='nfl_player_id_1']
train[[i+'_diff' for i in cols]] = np.abs(train[[i+'_1' for i in cols]].values - train[[i+'_2' for i in cols]].values)
test[[i+'_diff' for i in cols]] = np.abs(test[[i+'_1' for i in cols]].values - test[[i+'_2' for i in cols]].values)
feature_cols += [i+'_diff' for i in cols]

cols = ['x_position', 'y_position', 'speed', 'distance', 'direction', 'orientation', 'acceleration', 'sa']
train[[i+'_prod' for i in cols]] = train[[i+'_1' for i in cols]].values * train[[i+'_2' for i in cols]].values
test[[i+'_prod' for i in cols]] = test[[i+'_1' for i in cols]].values * test[[i+'_2' for i in cols]].values
feature_cols += [i+'_prod' for i in cols]

print('number of features : ',len(feature_cols))
print('number of train data : ',len(train))

number of features :  130
number of train data :  660560


# Train & Infer XGBoost model

In [ ]:
# ==============================
# training & inference
# ==============================

cfg.folds = get_groupkfold(train, 'contact', 'game_play', cfg.num_fold)
cfg.folds.to_csv(os.path.join(cfg.EXP_PREDS, 'folds.csv'), index=False)

oof_pred[np.where(cond_dis_train)] = fit_xgboost(cfg, train[feature_cols], train['contact'], 
                                              cfg.xgb_params, add_suffix="_xgb_1st")
np.save('oof_pred.npy',oof_pred)
sub_pred = pred_xgboost(test.loc[cond_dis_test, feature_cols], cfg.EXP_MODEL, add_suffix="_xgb_1st")

0    119036
1     13030
Name: contact, dtype: int64

[0]	train-auc:0.90405	eval-auc:0.90469
[100]	train-auc:0.94383	eval-auc:0.94051
[200]	train-auc:0.95816	eval-auc:0.94846
[300]	train-auc:0.96509	eval-auc:0.95135
[400]	train-auc:0.96982	eval-auc:0.95283
[500]	train-auc:0.97312	eval-auc:0.95355
[600]	train-auc:0.97581	eval-auc:0.95406
[700]	train-auc:0.97811	eval-auc:0.95433
[800]	train-auc:0.98021	eval-auc:0.95445
[900]	train-auc:0.98196	eval-auc:0.95456
[1000]	train-auc:0.98367	eval-auc:0.95484
[1100]	train-auc:0.98522	eval-auc:0.95485
[1200]	train-auc:0.98678	eval-auc:0.95497
[1281]	train-auc:0.98782	eval-auc:0.95495
Performance of the prediction: 0.95495



0    119555
1     12629
Name: contact, dtype: int64

[0]	train-auc:0.90698	eval-auc:0.90237
[100]	train-auc:0.94524	eval-auc:0.93513
[200]	train-auc:0.95916	eval-auc:0.94254
[300]	train-auc:0.96635	eval-auc:0.94429
[400]	train-auc:0.97078	eval-auc:0.94503
[500]	train-auc:0.97433	eval-auc:0.94531
[600]	train-auc:0.97704	eval-auc:0.94559
[700]	train-auc:0.97969	eval-auc:0.94573
[800]	train-auc:0.98170	eval-auc:0.94591
[900]	train-auc:0.98343	eval-auc:0.94594
[958]	train-auc:0.98439	eval-auc:0.94599
Performance of the prediction: 0.94599



0    118545
1     13620
Name: contact, dtype: int64

[0]	train-auc:0.90905	eval-auc:0.89162
[100]	train-auc:0.94630	eval-auc:0.92961
[200]	train-auc:0.96029	eval-auc:0.93896
[300]	train-auc:0.96743	eval-auc:0.94168
[400]	train-auc:0.97172	eval-auc:0.94285
[500]	train-auc:0.97492	eval-auc:0.94321
[600]	train-auc:0.97756	eval-auc:0.94340
[700]	train-auc:0.97999	eval-auc:0.94367
[800]	train-auc:0.98209	eval-auc:0.94365
[821]	train-auc:0.98251	eval-auc:0.94357
Performance of the prediction: 0.94357



0    119615
1     12566
Name: contact, dtype: int64

[0]	train-auc:0.90809	eval-auc:0.89748
[100]	train-auc:0.94664	eval-auc:0.92884
[200]	train-auc:0.96097	eval-auc:0.93529
[300]	train-auc:0.96753	eval-auc:0.93755
[400]	train-auc:0.97182	eval-auc:0.93838
[500]	train-auc:0.97511	eval-auc:0.93904
[600]	train-auc:0.97765	eval-auc:0.93957
[700]	train-auc:0.98023	eval-auc:0.93982
[800]	train-auc:0.98214	eval-auc:0.94000
[900]	train-auc:0.98375	eval-auc:0.94028
[1000]	train-auc:0.98559	eval-auc:0.94031
[1100]	train-auc:0.98677	eval-auc:0.94051
[1200]	train-auc:0.98810	eval-auc:0.94042
[1213]	train-auc:0.98822	eval-auc:0.94045
Performance of the prediction: 0.94045



0    119497
1     12467
Name: contact, dtype: int64

[0]	train-auc:0.90750	eval-auc:0.90304
[100]	train-auc:0.94552	eval-auc:0.93431
[200]	train-auc:0.96044	eval-auc:0.94068
[300]	train-auc:0.96768	eval-auc:0.94294
[400]	train-auc:0.97192	eval-auc:0.94344
[500]	train-auc:0.97523	eval-auc:0.94379
[600]	train-auc:0.97809	eval-auc:0.94398
[700]	train-auc:0.98046	eval-auc:0.94391
[708]	train-auc:0.98070	eval-auc:0.94396
Performance of the prediction: 0.94396

All Performance of the prediction: 0.94593


# Submission

In [ ]:
# ==============================
# optimize
# ==============================
def func(x_list):
    score = matthews_corrcoef(train_y, oof_pred>x_list[0])
    return -score

x0 = [0.5]
result = minimize(func, x0,  method="nelder-mead")
cfg.threshold = result.x[0]
print("score:", round(matthews_corrcoef(train_y, oof_pred>cfg.threshold), 5))
print("threshold", round(cfg.threshold, 5))
del train
gc.collect()

test = add_contact_id(test)
test['contact'] = 0
test.loc[cond_dis_test, 'contact'] = (sub_pred > cfg.threshold).astype(int)
test[['contact_id', 'contact']].to_csv('submission.csv', index=False)
display(test[['contact_id', 'contact']].head())

score: 0.65938
threshold 0.32041


,contact_id,contact
0,58168_003392_0_37084_37211,0
1,58168_003392_0_37084_38556,0
2,58168_003392_0_37084_38567,0
3,58168_003392_0_37084_38590,0
4,58168_003392_0_37084_39947,0
